# Анализ базы данных StackOverflow 

**Цель:** применить продвинутый SQL для анализа базы данных StackOverflow.

**Описание данных:**

Таблица stackoverflow.badges. Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про PostgreSQL, может получить значок postgresql. 
* id - Идентификатор значка, первичный ключ таблицы
* name - Название значка
* user_id - Идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users
* creation_date	- Дата присвоения значка

Таблица stackoverflow.post_types. Содержит информацию о типе постов. Их может быть два: Question — пост с вопросом, Answer — пост с ответом.
* id - Идентификатор поста, первичный ключ таблицы
* type	- Тип поста

Таблица stackoverflow.posts. Содержит информацию о постах.
* id	- Идентификатор поста, первичный ключ таблицы
* title	- Заголовок поста
* creation_date	- Дата создания поста
* favorites_count	- Число, которое показывает, сколько раз пост добавили в «Закладки»
* last_activity_date - Дата последнего действия в посте, например комментария
* last_edit_date	- Дата последнего изменения поста
* user_id	- Идентификатор пользователя, который создал пост, внешний ключ к таблице users
* parent_id	- Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом
* post_type_id	- Идентификатор типа поста, внешний ключ к таблице post_types
* score	- Количество очков, которое набрал пост
* views_count	- Количество просмотров

Таблица stackoverflow.users. Содержит информацию о пользователях.
* id	- Идентификатор пользователя, первичный ключ таблицы
* creation_date	- Дата регистрации пользователя
* display_name	- Имя пользователя
* last_access_date	- Дата последнего входа
* location	- Местоположение
* reputation	- Очки репутации, которые получают за хорошие вопросы и полезные ответы
* views	- Число просмотров профиля пользователя

Таблица stackoverflow.vote_types. Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов бывает несколько: 
* UpMod — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.
* DownMod — такую отметку получают посты, которые показались пользователям наименее полезными.
* Close — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.
* Offensive — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.
* Spam — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.

* id	- Идентификатор типа голоса, первичный ключ
* name	- Название метки

Таблица stackoverflow.votes. Содержит информацию о голосах за посты. 
* id	- Идентификатор голоса, первичный ключ
* post_id	- Идентификатор поста, внешний ключ к таблице posts
* user_id	- Идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users
* bounty_amount	- Сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту
* vote_type_id	- Идентификатор типа голоса, внешний ключ к таблице vote_types
* creation_date	- Дата назначения голоса



In [4]:
import pandas as pd

В данной работе не будем раскрывать путь к базе данных, а также для простоты восприятия не будем использовать для каждого вывода ячейки конструкцию ниже:

In [ ]:
query = '''
           
        '''

pd.io.sql.read_sql(query, con = engine) 

**Часть 1**

1.Найдем количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(id)
FROM stackoverflow.posts 
WHERE (post_type_id=1 AND score>300) OR favorites_count>=100;

2.Определим сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно, результат округлим до целого числа.

In [ ]:
SELECT ROUND(AVG(count_id))
FROM (
SELECT CAST(DATE_TRUNC('day', creation_date) AS date) AS day,
       COUNT(id) AS count_id
FROM stackoverflow.posts
WHERE post_type_id=1 AND CAST(DATE_TRUNC('day', creation_date) AS date) BETWEEN '2008-11-01' and '2008-11-18' 
GROUP BY day
) AS posts_grouped; 

3.Посмотрим, сколько пользователей получили значки сразу в день регистрации, выведем количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT b.user_id)
FROM stackoverflow.badges b 
INNER JOIN  stackoverflow.users u  ON b.user_id=u.id
WHERE CAST(b.creation_date AS date)=CAST(u.creation_date AS date);

4.Проанализируем сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос.

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM  stackoverflow.posts p 
INNER JOIN stackoverflow.users u  ON p.user_id=u.id
INNER JOIN stackoverflow.votes v ON p.id=v.post_id
WHERE p.score>=1 AND u.display_name='Joel Coehoorn';

5.Выгрузим все поля таблицы vote_types. Добавим к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблицу  отсортируем по полю id.

In [ ]:
SELECT *,
       RANK() OVER (ORDER BY id DESC) 
FROM stackoverflow.vote_types
ORDER BY id;

6.Отберем 10 пользователей, которые поставили больше всего голосов типа Close. Отобразим таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируем данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT u.id,
       COUNT(v.id)
FROM stackoverflow.users u
LEFT JOIN stackoverflow.votes v ON u.id=v.user_id
LEFT JOIN stackoverflow.vote_types vt ON v.vote_type_id=vt.id
WHERE vt.name='Close'
GROUP BY u.id
ORDER BY  COUNT(v.id) DESC
LIMIT 10;

7.Отберем 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразим несколько полей:
* идентификатор пользователя;
* число значков;
* место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвоим одно и то же место в рейтинге.

Отсортируем записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT u.id,
       COUNT(b.id),
       DENSE_RANK() OVER(ORDER BY COUNT(b.id)DESC)
FROM stackoverflow.users u
LEFT JOIN stackoverflow.badges b ON u.id=b.user_id
WHERE CAST(DATE_TRUNC('day', b.creation_date) AS date) BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY u.id
ORDER BY COUNT(b.id) DESC, 
         u.id
LIMIT 10;

8.Посмотрим сколько в среднем очков получает пост каждого пользователя и сформируем таблицу из следующих полей:
* заголовок поста;
* идентификатор пользователя;
* число очков поста;
 *среднее число очков пользователя за пост, округлённое до целого числа.
    
Не будем учитывать  посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER(PARTITION BY user_id))
FROM stackoverflow.posts 
WHERE score!=0 AND title IS NOT NULL;

9.Отобразим заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT p.title
FROM stackoverflow.posts p
WHERE p.user_id IN
(SELECT b.user_id
FROM stackoverflow.badges b
LEFT JOIN stackoverflow.posts p ON p.creation_date=b.creation_date
GROUP BY b.user_id
HAVING COUNT (DISTINCT b.id)>1000) AND title IS NOT NULL;

10.Напишем запрос, который выгрузит данные о пользователях из США (англ. United States). Разделим пользователей на три группы в зависимости от количества просмотров их профилей:
* пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
* пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
* пользователям с числом просмотров меньше 100 — группу 3.

Отобразим в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
       views,
       CASE 
           WHEN views >=350 THEN 1                  
           WHEN views >=100 AND views<350 THEN 2           
           WHEN views <100 THEN 3  
       END
FROM stackoverflow.users 
WHERE location LIKE '%United States%' AND views!= 0;

11.Дополним предыдущий запрос. Отобразим лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведем поля с идентификатором пользователя, группой и количеством просмотров. Отсортируем таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
SELECT id,
       group_number,
       views
FROM (SELECT *,
             MAX(views) OVER (PARTITION BY group_number) AS max_value
      FROM (SELECT id,
                   views,
                   CASE
                       WHEN views >= 350 THEN 1
                       WHEN views >= 100 THEN 2
                       ELSE 3
                   END AS group_number
            FROM stackoverflow.users AS u
            WHERE location LIKE '%United States%' AND views> 0) AS a) AS foo
WHERE max_value = views
ORDER BY views DESC, id;

12.Посчитаем ежедневный прирост новых пользователей в ноябре 2008 года. Сформируем таблицу с полями:
номер дня;
* число пользователей, зарегистрированных в этот день;
* сумму пользователей с накоплением.

In [ ]:
WITH users_1 AS (SELECT EXTRACT(DAY FROM creation_date) AS day_reg,
                        COUNT(id) AS count_id
                 FROM stackoverflow.users u
                 WHERE EXTRACT(MONTH FROM creation_date) = 11
                           AND EXTRACT(YEAR FROM creation_date) = 2008
                 GROUP BY day_reg)
SELECT *,
       SUM(count_id) OVER (ORDER BY day_reg)
FROM users_1;

13.Для каждого пользователя, который написал хотя бы один пост, найдем интервал между регистрацией и временем создания первого поста. Отобразим:
* идентификатор пользователя;
* разницу во времени между регистрацией и первым постом.

In [ ]:
SELECT DISTINCT users.id,
       MIN(posts.creation_date) OVER(PARTITION BY posts.user_id ORDER BY posts.creation_date)-users.creation_date
FROM  stackoverflow.posts
JOIN stackoverflow.users ON posts.user_id=users.id
ORDER BY 1;

**Часть 2**

1.Выведем общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируем по убыванию общего количества просмотров.

In [ ]:
SELECT CAST(DATE_TRUNC('month', creation_date)AS date) AS month,
       SUM(views_count) AS vc
FROM stackoverflow.posts
GROUP BY month
ORDER BY vc DESC;

2.Выведем имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не будем учитывать. Для каждого имени пользователя выведем количество уникальных значений user_id. Отсортируем результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
       COUNT(DISTINCT user_id)
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
JOIN stackoverflow.users u ON u.id = p.user_id
WHERE DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date)
  AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'
  AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(*) > 100;

3.Выведем количество постов за 2008 год по месяцам. Отберем посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируем таблицу по значению месяца по убыванию.

In [ ]:
SELECT CAST(DATE_TRUNC('month', p.creation_date)AS date) AS month,
       COUNT(p.id) AS posts_count
FROM stackoverflow.posts p
WHERE p.user_id IN (
    SELECT u.id
FROM stackoverflow.posts p
LEFT JOIN stackoverflow.users u ON p.user_id=u.id
WHERE EXTRACT(YEAR FROM p.creation_date)= 2008 AND EXTRACT(MONTH FROM p.creation_date)=12 AND EXTRACT(YEAR FROM u.creation_date)= 2008 AND EXTRACT(MONTH FROM u.creation_date)=09
) 

GROUP BY month
ORDER BY month DESC;

4.Используя данные о постах, выведем несколько полей:
* идентификатор пользователя, который написал пост;
* дата создания поста;
* количество просмотров у текущего поста;
* сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT p.user_id,
       p.creation_date,
       p.views_count,
       SUM(p.views_count) OVER(PARTITION BY p.user_id ORDER BY p.creation_date)
FROM stackoverflow.posts p
ORDER BY p.user_id;

5.Проанализируем сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой. Для каждого пользователя отберем дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число, округлим результат.

In [ ]:
WITH temp_table AS
  (SELECT distinct user_id,
          DATE_TRUNC('day', creation_date) AS day
   FROM stackoverflow.posts
   WHERE DATE_TRUNC('day',creation_date) >= '2008-12-01'
     AND DATE_TRUNC('day',creation_date) <= '2008-12-07')
SELECT ROUND(AVG(days_count)) AS result
FROM
  (SELECT user_id,
          COUNT(day) AS days_count
   FROM temp_table
   GROUP BY user_id
   ORDER BY user_id) AS temp_table2;

6.Посмотрим на сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года. Отобразим таблицу со следующими полями:
* номер месяца;
* количество постов за месяц;
* процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлим значение процента до двух знаков после запятой.

При делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведем делимое в тип numeric.

In [ ]:
WITH t AS (
SELECT EXTRACT(MONTH FROM p.creation_date) AS months,
       COUNT(p.id) AS posts_count
FROM stackoverflow.posts p
WHERE CAST(p.creation_date AS date) BETWEEN '2008-09-01'  AND '2008_12-31'
GROUP BY months
)

SELECT *,
       ROUND(((posts_count::numeric / LAG(posts_count) OVER (ORDER BY months)) - 1) * 100, 2) AS post_growth
FROM t;

7.Выгрузим данные активности пользователя, который опубликовал больше всего постов за всё время. Выведем данные за октябрь 2008 года в таком виде:
* номер недели;
* дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
WITH t AS (
SELECT EXTRACT(WEEK FROM p.creation_date) AS week,
       CAST(p.creation_date AS timestamp) AS dates
FROM stackoverflow.posts p
WHERE p.creation_date BETWEEN '2008-10-01 00:00:00' and '2008-10-31 23:59:59' AND p.user_id IN (
    SELECT user_id      
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY COUNT(id) DESC
LIMIT 1
)
)

SELECT DISTINCT week,
       LAST_VALUE(t.dates) OVER (PARTITION BY week ORDER BY week)
FROM t
ORDER BY week;       